In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from xgboost import XGBClassifier
from sklearn.svm import SVC
import pickle
import joblib 
from joblib import Parallel, delayed

In [2]:
data = pd.read_csv("D:\DEEPALI\PROJECT_SEM_IV_Medical\Data\Excel data and code\Diabetes\diabetes_prediction_dataset.csv")
data.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


In [4]:
data['smoking_history'].value_counts()

No Info        35816
never          35095
former          9352
current         9286
not current     6447
ever            4004
Name: smoking_history, dtype: int64

In [5]:
dict_map = {'No Info':'No Information','never':'Never',
            'former':'Not Current', 'current': 'Current','not current': 'Not Current','ever':'Not Current' }
updateSer = data['smoking_history'].map(dict_map)
data['smoking_history'] = updateSer

In [6]:
data['smoking_history'].value_counts()

No Information    35816
Never             35095
Not Current       19803
Current            9286
Name: smoking_history, dtype: int64

In [7]:
data.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,Never,25.19,6.6,140,0
1,Female,54.0,0,0,No Information,27.32,6.6,80,0
2,Male,28.0,0,0,Never,27.32,5.7,158,0
3,Female,36.0,0,0,Current,23.45,5.0,155,0
4,Male,76.0,1,1,Current,20.14,4.8,155,0


In [8]:
def process(df):
    df = pd.get_dummies(df, columns = ['smoking_history','gender'])
    return df

data = process(data)
data.head()

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes,smoking_history_Current,smoking_history_Never,smoking_history_No Information,smoking_history_Not Current,gender_Female,gender_Male,gender_Other
0,80.0,0,1,25.19,6.6,140,0,0,1,0,0,1,0,0
1,54.0,0,0,27.32,6.6,80,0,0,0,1,0,1,0,0
2,28.0,0,0,27.32,5.7,158,0,0,1,0,0,0,1,0
3,36.0,0,0,23.45,5.0,155,0,1,0,0,0,1,0,0
4,76.0,1,1,20.14,4.8,155,0,1,0,0,0,0,1,0


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 14 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   age                             100000 non-null  float64
 1   hypertension                    100000 non-null  int64  
 2   heart_disease                   100000 non-null  int64  
 3   bmi                             100000 non-null  float64
 4   HbA1c_level                     100000 non-null  float64
 5   blood_glucose_level             100000 non-null  int64  
 6   diabetes                        100000 non-null  int64  
 7   smoking_history_Current         100000 non-null  uint8  
 8   smoking_history_Never           100000 non-null  uint8  
 9   smoking_history_No Information  100000 non-null  uint8  
 10  smoking_history_Not Current     100000 non-null  uint8  
 11  gender_Female                   100000 non-null  uint8  
 12  gender_Male      

In [10]:
# dividing data
x = data.drop('diabetes', axis =1)
y = data['diabetes']

In [11]:
x_tr, x_test, y_tr, y_test = train_test_split(x, y, test_size = 0.3, stratify = y)
x_tr.shape, x_test.shape, y_tr.shape, y_test.shape

((70000, 13), (30000, 13), (70000,), (30000,))

In [12]:
# XGboost
diab_xg1 = XGBClassifier()
diab_xg1.fit(x_tr, y_tr)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [13]:
pred_xg1 = diab_xg1.predict(x_test)

In [14]:
cm = confusion_matrix(y_test, pred_xg1)
cm

array([[27368,    82],
       [  858,  1692]], dtype=int64)

In [15]:
print(classification_report(y_test, pred_xg1))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98     27450
           1       0.95      0.66      0.78      2550

    accuracy                           0.97     30000
   macro avg       0.96      0.83      0.88     30000
weighted avg       0.97      0.97      0.97     30000



In [16]:
# save the model
#joblib.dump(diab_xg, 'diab_xg1.pkl')

In [18]:
joblib.dump(diab_xg1, 'diab_xg1.pkl')


['diab_xg1.pkl']

In [19]:
tr_pred = diab_xg1.predict(x_tr)

In [20]:
print(classification_report(y_tr, tr_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     64050
           1       0.99      0.74      0.84      5950

    accuracy                           0.98     70000
   macro avg       0.98      0.87      0.92     70000
weighted avg       0.98      0.98      0.98     70000



In [26]:
model = joblib.load('diab_xg1.pkl')

In [27]:
new_data = np.array([[21.0, 0, 0, 35.6, 4.8, 100, 0, 0, 0, 1, 1, 0, 0]])

In [28]:
pred=  model.predict(new_data)
pred

array([0])